In [6]:
import pandas as pd

title_akas = pd.read_csv('../downloads/title.akas.tsv', delimiter='\t', dtype={'isOriginalTitle': object})
title_basics = pd.read_csv('../downloads/title.basics.tsv', delimiter='\t', dtype={'isAdult': object})
title_crew = pd.read_csv('../downloads/title.crew.tsv', delimiter='\t', dtype={'directors': 'str'})
title_ratings = pd.read_csv('../downloads/title.ratings.tsv', delimiter='\t')
name_basics = pd.read_csv('../downloads/name.basics.tsv', delimiter='\t')

In [2]:
# auxiliary functions 

def drop_columns(df, columns): return df.drop(labels=columns, axis=1)

In [3]:
def get_director_names(row):
  if row['directors'] == '\\N':
    row['directors'] = None
    return
  
  directors_code = row['directors'].split(",")
  directors = ''

  for code in directors_code:
    directors += name_basics[name_basics['nconst'] == code]['primaryName']

  row['directors'] = directors

In [4]:
# filter by titleType

title_basics = title_basics[(title_basics['titleType'] == 'movie') | (title_basics['titleType'] == 'tvMovie')]

# filter by isOriginalTitle
#FIXME
title_akas['isOriginalTitle'] = title_akas['isOriginalTitle'].replace('NaN', None)
title_akas = title_akas[title_akas['isOriginalTitle'] == 1 | pd.isna(title_akas['isOriginalTitle'])]

In [5]:
# drop useless columns 

title_akas = drop_columns(title_akas, ['ordering', 'types', 'attributes', 'title'])
title_basics = drop_columns(title_basics, ['endYear', 'titleType'])
title_crew = drop_columns(title_crew, ['writers'])
title_ratings = drop_columns(title_ratings, ['numVotes'])
name_basics = drop_columns(name_basics, ['birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'])

In [6]:
# If it is to remove movies before a certain year
# Remove from title_basics

In [7]:
df = title_basics.merge(
  title_crew, on='tconst', how='left').merge(
    title_ratings, on='tconst', how='left')

In [8]:
#TODO get movies with same tconst as title_basics
teste2 = title_basics.merge(
  title_akas, left_on='tconst', right_on='titleId', how='left')

In [11]:
#REVIEW takes too long
df.apply(lambda row: get_director_names(row), axis=1)
df

KeyboardInterrupt: 

# TODO:
- remover filmes mais antigos que x data
- filtrar filmes apenas em en
- remover filmes sem sinopse (isto é quando formos fazer o webscrapping ig)
